In [1]:
using LinearAlgebra
using Plots
using StaticArrays
using FastPow

In [67]:
function calc_forces(r, N, max_distance)
    a = zeros(N,2)
    V_LJ = 0
    for j ∈ 1:N
        for k ∈ j+1:N
            if j != k
            Δr = r[j,:] .- r[k,:]
            r_norm = norm(Δr, 1)
            r_norm_sq = r_norm^2
                if r_norm < max_distance && r_norm > 0
                    @fastpow a[j,:] += (2/r_norm_sq^6  -1/r_norm_sq^3)*Δr/r_norm_sq
                    a[k,:] -= a[j,:]
                    @fastpow V_LJ += 4*(r_norm_sq^(-6) - r_norm_sq^(-3))                    
                end
            end
        end
    end
    return a, V_LJ
end

function sum_forces(r, v, a, dt, N, max_distance)
    a_prev = a
    r += v*dt + 0.5*a*dt^2
    a, V_LJ = calc_forces(r, N, max_distance)
    v += 0.5*(a+a_prev)*dt
    K = 1/2*norm(v)^2
    
    return r, v, a, V_LJ, K    
end


function time_iteration(dt, N, steps, r, v, max_distance)
    a = zeros(steps, N, 2)
    V_LJ = zeros(steps)
    K = zeros(steps)
    for i in 1:steps-1
        r[i+1,:,:], v[i+1,:,:], a[i+1,:,:], V_LJ[i+1], K[i+1] = sum_forces(r[i,:,:], v[i,:,:], a[i,:,:], dt, N, max_distance)
    end
    return r, v, V_LJ, K
end


function initial_particles(N, l, u)
    # Generate N particles randomly in a box with side lengths l
    # ensuring minimum distance u between each point

    # Initialize an empty array to store the particles
    particles = zeros(2,N)
    println(size(particles[:,1]))
    count = 0

    # Generate random coordinates for each point
    while count < N
        x = rand(-l/2:l/1000:l/2)
        y = rand(-l/2:l/1000:l/2)

        # Check the minimum distance u from each existing point
        valid = true
        for i in 1:count
            if norm([x,y] .- particles[i,:]) < u
                valid = false
                break
            end
        end

        # Add the point to the matrix if it satisfies the minimum distance constraint
        if valid
            count += 1
            particles[:, count] = [x, y]
        end
    end
    

    return particles
end


function Wave(N, l, u, t, dt)
    steps = Int(t/dt)
    r_0 = initial_particles(N, l, u)
    v_0 = copy(r_0)
    for velocity_vector in eachrow(r_0)
        if norm(velocity_vector) > 1
            veclocity_vector = [0,0]
        end
    end
    
    
    r = zeros(steps, N, 2)
    v = zeros(steps, N, 2)
    # print(r_0)
    # print(r)
    r[1,:,:] = r_0
    v[1,:,:] = v_0
    return r, v
end 

# struct Simulation
#     N:: Int
#     l:: Float64
#     t:: Float64
#     u:: Float64
#     dt:: Float64
#     steps::Int = (t/dt)
# end



Wave (generic function with 2 methods)

In [68]:
N = 300
l = 20
t = 0.125
dt = 5*10^(-4)
min_distance = 0.5
max_distance = 3
steps = Int(t/dt)
r_0, v_0 = Wave(N,l,min_distance,t,dt)

@time r, v, V_JL, kinetic_energy = time_iteration(dt, N, steps, r_0, v_0, max_distance)

println(typeof(r[1,:,:]))
println(size(r))
println(sizeof(V_JL))
print(sizeof(kinetic_energy))


(2,)


DimensionMismatch: DimensionMismatch: arrays could not be broadcast to a common size; got a dimension with lengths 2 and 300

In [69]:
function Circle(r)
    x = LinRange(0,2π,100)
    return r*sin.(x) , r*cos.(x)
end

Circle (generic function with 1 method)

In [70]:
function plot_particles(r)
    @gif for i ∈ 1:steps
        rx = r[i,:,1]
        ry = r[i,:,2]
        scatter(rx,ry, xlim=(-l/2,l/2), ylim=(-l/2,l/2), markersize=2, legend=false)
        plot!(Circle(i/10), xlim=(-l/2,l/2), ylim=(-l/2,l/2))
    end
end


function plot_energy(U, K, T, steps)
    t = LinRange(0, T, steps)
    plot(t, U, label='U')
    plot!(t, K, label='K')
    plot!(t, (U+K), label='E')
end

plot_energy (generic function with 1 method)

In [71]:
plot_particles(r)

UndefVarError: UndefVarError: r not defined

In [72]:
plot_energy(V_JL, kinetic_energy, t, steps)

UndefVarError: UndefVarError: V_JL not defined

In [73]:
N = 10
@time r_0 = SVector{N}([SVector{2}(rand(2)) for i in 1:N])
@time r_1 = [rand(2) for i in 1:N]


SVector{3}([SVector{250} SVector{200} SVector{2}])

  0.184602 seconds (55.17 k allocations: 2.813 MiB, 99.47% compilation time)


  0.061445 seconds (53.04 k allocations: 2.598 MiB, 99.14% compilation time)


3-element SVector{3, UnionAll} with indices SOneTo(3):
 SVector{250} (alias for SArray{Tuple{250}, T, 1, 250} where T)
 SVector{200} (alias for SArray{Tuple{200}, T, 1, 200} where T)
 SVector{2} (alias for SArray{Tuple{2}, T, 1, 2} where T)